
#**Multiclass Venn-ABERS calibration & Keratoconus detection**


- Dataset: https://www.kaggle.com/datasets/imenbakir/keratoconus-stage-detection
- An introduction to Venn & Venn-Abers predictors: https://www.youtube.com/watch?v=KsQpkjl7u1w&ab_channel=KhuongAnNguyen
- A tutorial:
https://cml.rhul.ac.uk/people/ptocca/HomePage/Toccaceli_CP___Venn_Tutorial.pdf
- Awesome conformal prediction: https://github.com/valeman/awesome-conformal-prediction


In [1]:
!git clone https://github.com/ip200/venn-abers


Cloning into 'venn-abers'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 131 (delta 64), reused 95 (delta 38), pack-reused 0
Receiving objects: 100% (131/131), 846.56 KiB | 11.44 MiB/s, done.
Resolving deltas: 100% (64/64), done.


#Libraries

In [46]:
import pandas as pd
import numpy as np
from scipy.io import arff
from sklearn.metrics import log_loss, brier_score_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from sklearn.calibration import CalibratedClassifierCV

from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings('ignore')

from venn_abers import VennAbersCalibrator

#Load dataset

In [10]:
data= pd.read_csv('/content/dataset.csv')
labels=pd.read_csv('/content/labels.csv')

In [11]:
data.head()

,Unnamed: 0,idEye,Ks,Ks.Axis,Kf,Kf.Axis,AvgK,CYL,AA,Ecc.9.0mm.,...,coma.5,coma.axis.5,SA.C40..5,S35.coma.like..5,S46.sph..like..5,HOAs.S3456..5,AA.5,En.Anterior.,ESI.Anterior.,ESI.Posterior.
0,9,1OS(Left),44.53,21,39.22,111,41.87,5.32,86.7,0.91,...,3.131,97,-0.722,3.350,1.053,3.512,99,Enable,45,27
1,10,1OD(Right),43.84,39,42.46,129,43.15,1.38,88.2,0.65,...,0.575,97,0.085,0.921,0.290,0.966,100,Enable,0,0
2,39,2OD(Right),44.81,66,44.41,156,44.61,0.40,83.0,0.48,...,0.177,9,0.268,0.263,0.640,0.692,100,Enable,0,0
3,55,4OS(Left),44.00,51,42.31,141,43.15,1.69,97.3,0.60,...,0.492,275,-0.281,3.396,1.419,3.680,100,Enable,0,29
4,56,4OD(Right),45.42,26,45.20,116,45.31,0.22,93.3,0.69,...,0.571,85,0.109,0.691,0.181,0.714,100,Enable,0,7


In [12]:
labels.head()

,Unnamed: 0,Data.PLOS_One.idEye,clster_labels
0,1,1OS(Left),1
1,2,1OD(Right),2
2,3,2OD(Right),2
3,4,4OS(Left),1
4,5,4OD(Right),2


In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() #creating an instance of the label encoder called 'le'
data_le = data.copy(deep = True) #copying the dataframe
#deep=True : modifications to the data or indices of the copy will not be reflected in the original object
label = le.fit_transform(data_le['idEye'].values) #fit the label encoder with the 'fit_transform' function
label

array([1638, 1637, 1822, ..., 1593, 1596, 1595])

In [17]:
data_le['idEye']=label
data_le.head(10)

,Unnamed: 0,idEye,Ks,Ks.Axis,Kf,Kf.Axis,AvgK,CYL,AA,Ecc.9.0mm.,...,coma.5,coma.axis.5,SA.C40..5,S35.coma.like..5,S46.sph..like..5,HOAs.S3456..5,AA.5,En.Anterior.,ESI.Anterior.,ESI.Posterior.
0,9,1638,44.53,21,39.22,111,41.87,5.32,86.7,0.91,...,3.131,97,-0.722,3.350,1.053,3.512,99,Enable,45,27
1,10,1637,43.84,39,42.46,129,43.15,1.38,88.2,0.65,...,0.575,97,0.085,0.921,0.290,0.966,100,Enable,0,0
2,39,1822,44.81,66,44.41,156,44.61,0.40,83.0,0.48,...,0.177,9,0.268,0.263,0.640,0.692,100,Enable,0,0
3,55,2204,44.00,51,42.31,141,43.15,1.69,97.3,0.60,...,0.492,275,-0.281,3.396,1.419,3.680,100,Enable,0,29
4,56,2203,45.42,26,45.20,116,45.31,0.22,93.3,0.69,...,0.571,85,0.109,0.691,0.181,0.714,100,Enable,0,7
5,68,2399,62.98,68,42.51,158,52.74,20.47,95.5,0.25,...,3.791,45,-1.581,13.387,9.154,16.218,98,Enable,95,0
6,69,2398,44.23,85,42.86,175,43.54,1.37,96.7,0.43,...,0.269,88,0.105,0.398,0.376,0.548,100,Enable,0,0
7,87,2592,44.61,170,43.75,80,44.18,0.86,97.9,0.46,...,0.085,20,0.375,0.437,0.360,0.566,100,Enable,0,52
8,88,2593,44.13,174,43.03,84,43.58,1.10,97.2,0.46,...,0.146,229,0.353,0.371,0.485,0.611,100,Enable,0,63
9,138,2968,45.83,165,45.37,75,45.60,0.46,96.8,0.80,...,0.219,74,0.269,0.330,0.315,0.457,100,Enable,0,24


In [22]:
data_le.drop('En.Anterior.', inplace=True, axis=1)
data_le.head()

,Unnamed: 0,idEye,Ks,Ks.Axis,Kf,Kf.Axis,AvgK,CYL,AA,Ecc.9.0mm.,...,HOAs.S3456..4,coma.5,coma.axis.5,SA.C40..5,S35.coma.like..5,S46.sph..like..5,HOAs.S3456..5,AA.5,ESI.Anterior.,ESI.Posterior.
0,9,1638,44.53,21,39.22,111,41.87,5.32,86.7,0.91,...,1.057,3.131,97,-0.722,3.350,1.053,3.512,99,45,27
1,10,1637,43.84,39,42.46,129,43.15,1.38,88.2,0.65,...,0.283,0.575,97,0.085,0.921,0.290,0.966,100,0,0
2,39,1822,44.81,66,44.41,156,44.61,0.40,83.0,0.48,...,0.456,0.177,9,0.268,0.263,0.640,0.692,100,0,0
3,55,2204,44.00,51,42.31,141,43.15,1.69,97.3,0.60,...,1.250,0.492,275,-0.281,3.396,1.419,3.680,100,0,29
4,56,2203,45.42,26,45.20,116,45.31,0.22,93.3,0.69,...,0.387,0.571,85,0.109,0.691,0.181,0.714,100,0,7


In [24]:
kk = labels['clster_labels']
df=data_le.join(kk)
df

,Unnamed: 0,idEye,Ks,Ks.Axis,Kf,Kf.Axis,AvgK,CYL,AA,Ecc.9.0mm.,...,coma.5,coma.axis.5,SA.C40..5,S35.coma.like..5,S46.sph..like..5,HOAs.S3456..5,AA.5,ESI.Anterior.,ESI.Posterior.,clster_labels
0,9,1638,44.53,21,39.22,111,41.87,5.32,86.7,0.91,...,3.131,97,-0.722,3.350,1.053,3.512,99,45,27,1
1,10,1637,43.84,39,42.46,129,43.15,1.38,88.2,0.65,...,0.575,97,0.085,0.921,0.290,0.966,100,0,0,2
2,39,1822,44.81,66,44.41,156,44.61,0.40,83.0,0.48,...,0.177,9,0.268,0.263,0.640,0.692,100,0,0,2
3,55,2204,44.00,51,42.31,141,43.15,1.69,97.3,0.60,...,0.492,275,-0.281,3.396,1.419,3.680,100,0,29,1
4,56,2203,45.42,26,45.20,116,45.31,0.22,93.3,0.69,...,0.571,85,0.109,0.691,0.181,0.714,100,0,7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3157,12238,1591,46.60,86,45.79,176,46.19,0.81,97.0,0.75,...,0.384,339,0.164,0.583,0.259,0.638,100,0,10,2
3158,12239,1594,44.05,82,43.90,172,43.97,0.15,96.4,0.63,...,0.156,217,0.242,0.388,0.387,0.548,100,0,5,2
3159,12240,1593,44.12,3,43.47,93,43.80,0.66,96.6,0.60,...,0.179,337,0.348,0.313,0.384,0.495,100,0,0,2
3160,12241,1596,46.59,90,44.74,0,45.66,1.86,96.1,0.74,...,0.609,113,-0.124,1.092,0.842,1.379,100,0,0,2


In [25]:
df['clster_labels'].value_counts()


2    2595
1     264
3     221
4      82
Name: clster_labels, dtype: int64

In [26]:
df_train_cal, df_test = train_test_split(df, test_size = 800, random_state = 42, shuffle = False)
df_proper_train, df_cal = train_test_split(df_train_cal, test_size = 500, random_state = 42, shuffle = False)


In [27]:
X_train = df_train_cal.drop('clster_labels', axis=1)
y_train = df_train_cal['clster_labels']

X_proper_train = df_proper_train.drop('clster_labels', axis=1)
y_proper_train = df_proper_train['clster_labels']

X_cal = df_cal.drop('clster_labels', axis=1)
y_cal = df_cal['clster_labels']

X_test = df_test.drop('clster_labels', axis=1)
y_test = df_test['clster_labels']

In [28]:
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)


In [29]:
def brier_loss_calc(y_true, prob):
    return ((y_true - prob)**2).mean()


#Classifier comparison

In [78]:
clfs = {}
clfs['Naive Bayes'] = GaussianNB()
clfs['SVM'] = SVC(probability=True)
clfs['RF'] = RandomForestClassifier()
clfs['XGB'] = AdaBoostClassifier()
clfs['Logistic'] = LogisticRegression(max_iter=10000)
clfs['Neural Network'] =  MLPClassifier(max_iter=10000)


def run_multiclass_comparison(clf_name, clf):

    print(clf_name + ':')
    log_loss_list = []
    brier_loss_list = []
    acc_list = []
    ece_list = []

    print('base')
    clf.fit(X_train, y_train)
    p_pred = clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))

    print('sigmoid')
    clf.fit(X_proper_train, y_proper_train)
    cal_sigm = CalibratedClassifierCV(clf, method='sigmoid', cv='prefit')
    cal_sigm.fit(X_cal, y_cal)
    p_pred = cal_sigm.predict_proba(X_test)
    y_pred = cal_sigm.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))

    print('isotonic')
    cal_iso = CalibratedClassifierCV(clf, method='isotonic', cv='prefit')
    cal_iso.fit(X_cal, y_cal)
    p_pred = cal_iso.predict_proba(X_test)
    y_pred = cal_iso.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))

    print('sigmoid_cv')
    cal_sigm_cv = CalibratedClassifierCV(clf, method='sigmoid', cv=5)
    cal_sigm_cv.fit(X_train, y_train)
    p_pred = cal_sigm_cv.predict_proba(X_test)
    y_pred = cal_sigm_cv.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))

    print('isotonic_cv')
    cal_iso_cv = CalibratedClassifierCV(clf, method='isotonic', cv=5)
    cal_iso_cv.fit(X_train, y_train)
    p_pred = cal_iso_cv.predict_proba(X_test)
    y_pred = cal_iso_cv.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))

    print('ivap')
    va = VennAbersCalibrator(clf, inductive=True, cal_size=0.2, random_state=42)
    va.fit(np.asarray(X_train), np.asarray(y_train))
    p_pred_va = va.predict_proba(np.array(X_test))
    y_pred = va.predict(X_test, one_hot=False)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred_va))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred_va))

    print('cvap')
    va_cv = VennAbersCalibrator(clf, inductive=False, n_splits=5)
    va_cv.fit(np.asarray(X_train), np.asarray(y_train))
    p_pred_cv = va_cv.predict_proba(np.asarray(X_test))
    y_pred = va_cv.predict(X_test, one_hot=False)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred_cv))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred_cv))

    print('')

    df_ll = pd.DataFrame(columns=['Classifier', 'Uncalibrated', 'Platt', 'Isotonic', 'Platt-CV', 'Isotonic-CV', 'IVAP', 'CVAP'])
    df_ll.loc[0] =  [clf_name] + log_loss_list
    df_bl = pd.DataFrame(columns=['Classifier', 'Uncalibrated', 'Platt', 'Isotonic', 'Platt-CV', 'Isotonic-CV', 'IVAP', 'CVAP'])
    df_bl.loc[0] =  [clf_name] + brier_loss_list
    df_acc = pd.DataFrame(columns=['Classifier', 'Uncalibrated', 'Platt', 'Isotonic', 'Platt-CV', 'Isotonic-CV', 'IVAP', 'CVAP'])
    df_acc.loc[0] =  [clf_name] + acc_list

    return df_bl, df_ll, df_acc

In [89]:
results_brier = pd.DataFrame()
results_log = pd.DataFrame()
results_acc = pd.DataFrame()

for clf_name in clfs:
    scratch_b, scratch_l, scratch_acc = run_multiclass_comparison(clf_name, clfs[clf_name])
    results_brier = pd.concat((results_brier, scratch_b), ignore_index=True)
    results_log = pd.concat((results_log, scratch_l), ignore_index=True)
    results_acc = pd.concat((results_acc, scratch_acc), ignore_index=True)


Naive Bayes:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

SVM:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

RF:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

XGB:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

Logistic:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

Neural Network:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap



In [90]:
results_acc.set_index('Classifier', inplace=True)
results_acc.round(4)


,Uncalibrated,Platt,Isotonic,Platt-CV,Isotonic-CV,IVAP,CVAP
Classifier,,,,,,,
Naive Bayes,0.6262,0.8562,0.8988,0.9000,0.9012,0.0638,0.0638
SVM,0.8562,0.8812,0.9038,0.8850,0.8950,0.0538,0.0538
RF,0.9575,0.9600,0.9600,0.9600,0.9638,0.0538,0.8562
XGB,0.9075,0.8688,0.8750,0.8612,0.8775,0.8562,0.8562
Logistic,0.9150,0.9050,0.9088,0.9062,0.9188,0.8562,0.8562
Neural Network,0.8562,0.8362,0.8162,0.8975,0.9262,0.0262,0.0638


In [91]:
results_brier.set_index('Classifier', inplace=True)
results_brier.round(5)


,Uncalibrated,Platt,Isotonic,Platt-CV,Isotonic-CV,IVAP,CVAP
Classifier,,,,,,,
Naive Bayes,0.18565,0.04389,0.04301,0.04527,0.04471,0.04109,0.04155
SVM,0.02956,0.04267,0.03370,0.05790,0.05893,0.02918,0.02870
RF,0.01627,0.01484,0.01488,0.01425,0.01435,0.01558,0.01686
XGB,0.14717,0.04805,0.04473,0.04916,0.04335,0.01823,0.02100
Logistic,0.03713,0.03666,0.03642,0.03369,0.03050,0.03246,0.03069
Neural Network,0.06983,0.05140,0.05562,0.04246,0.03726,0.03392,0.03063


In [92]:
results_log.set_index('Classifier', inplace=True)
results_log.round(6)


,Uncalibrated,Platt,Isotonic,Platt-CV,Isotonic-CV,IVAP,CVAP
Classifier,,,,,,,
Naive Bayes,12.106936,0.328274,0.320011,0.336714,0.329641,0.298378,0.312777
SVM,0.267704,0.327700,0.490289,0.484499,0.487160,0.230038,0.234400
RF,0.123689,0.115365,0.138420,0.111030,0.137150,0.111201,0.134650
XGB,1.007932,0.375988,0.504077,0.388295,0.346721,0.152042,0.182379
Logistic,0.778025,0.275311,0.858824,0.258460,0.243185,0.290802,0.244317
Neural Network,2.959569,0.426839,0.485924,0.340631,0.301714,0.255354,0.225841


In [93]:
results_acc.mean()


Uncalibrated    0.853125
Platt           0.884583
Isotonic        0.893750
Platt-CV        0.901667
Isotonic-CV     0.913750
IVAP            0.318333
CVAP            0.458333
dtype: float64

In [94]:
results_brier.rank(axis=1).mean()


Uncalibrated    6.166667
Platt           4.666667
Isotonic        4.333333
Platt-CV        4.500000
Isotonic-CV     3.500000
IVAP            2.333333
CVAP            2.500000
dtype: float64

In [95]:
results_log.rank(axis=1).mean()


Uncalibrated    5.666667
Platt           4.000000
Isotonic        6.000000
Platt-CV        4.000000
Isotonic-CV     4.000000
IVAP            2.000000
CVAP            2.333333
dtype: float64

In [96]:
results_brier.mean()


Uncalibrated    0.080938
Platt           0.039584
Isotonic        0.038058
Platt-CV        0.040455
Isotonic-CV     0.038182
IVAP            0.028409
CVAP            0.028240
dtype: float64

In [97]:
results_log.mean()


Uncalibrated    2.873976
Platt           0.308246
Isotonic        0.466258
Platt-CV        0.319938
Isotonic-CV     0.307595
IVAP            0.222969
CVAP            0.222394
dtype: float64